In [4]:
import pandas as pd
import numpy as np
from pandas import Series,DataFrame

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
%matplotlib inline

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier

In [5]:
train_x = pd.read_csv("G:/Redhat_share/ML_micro/train_x.csv")
train_y = pd.read_csv("G:/Redhat_share/ML_micro/train_y.csv")
test_x = pd.read_csv("G:/Redhat_share/ML_micro/test_x.csv")
features_type = pd.read_csv("G:/Redhat_share/ML_micro/features_type.csv")
train_x.head()

,uid,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x1129,x1130,x1131,x1132,x1133,x1134,x1135,x1136,x1137,x1138
0,1792,5.04,5.77,6.04,3.91,0.66,1.04,1.83,5.41,5.45,...,1,1,20,4,1,7,15,6,6,6
1,4211,5.98,2.90,1.94,1.42,0.10,3.29,0.81,9.09,8.04,...,7,8,1,2,7,7,3,8,7,24
2,14658,7.04,8.92,3.24,0.01,0.00,4.50,0.00,8.41,6.97,...,6,6,1,1,6,8,6,7,4,19
3,17041,13.30,7.16,5.69,1.98,0.71,2.20,0.22,13.89,1.75,...,1,1,20,1,1,8,1,6,8,9
4,15765,5.24,5.09,5.31,1.05,0.00,4.24,0.34,5.25,5.59,...,7,1,3,1,1,8,31,1,6,14


In [6]:
train_y.head()

,uid,y
0,1792,1
1,4211,1
2,14658,1
3,17041,1
4,15765,1


In [ ]:
#数据整理工作


In [7]:
#去除数据中不相关元素
train_x.drop("uid",axis=1)
train_y = train_y["y"]

In [35]:
#逻辑回归
logreg = LogisticRegression()
logreg.fit(train_x,train_y)
pred_y = logreg.predict(test_x)
logreg.score(train_x,train_y)

0.8958666666666667

In [8]:
#随机森林
random_forest = RandomForestClassifier(n_estimators=200,max_features=0.3)
random_forest.fit(train_x,train_y)
pred_y = random_forest.predict(test_x)
random_forest.score(train_x,train_y)

1.0

In [10]:
#高斯朴素贝叶斯
gaussian = GaussianNB()
gaussian.fit(train_x, train_y)
pred_y = gaussian.predict(test_x)
gaussian.score(train_x, train_y)

0.2114

In [13]:
#支持向量机
svc = SVC()
svc.fit(train_x, train_y)
Y_pred = svc.predict(test_x)
svc.score(train_x, train_y)

1.0

In [17]:
#K最近邻算法
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(train_x, train_y)
pred_y = knn.predict(test_x)
knn.score(train_x, train_y)

0.9075333333333333

In [18]:
#将预测结果生成csv文件
submission = pd.DataFrame({
        "uid": test_x["uid"],
        "score": pred_y
    })
submission.to_csv('result.csv', index=False)